In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import tensorflow as tf
import matplotlib.pylab as plt
import numpy as np

In [4]:
import sys
sys.path.append('../../Main/')

In [5]:
from load_model import LoadModel
from stag_mag import calculate_stag_mag
from sigma_x import calculate_sigma_x
from energy_func import buildlattice, construct_mats

# Calculate Stag Mags & Sig Xs

In [6]:
deltas = np.arange(-1.545,13.455,0.5)

In [14]:
def calculate_observables(loadmodel_fxn,path_to_config,path_to_ckpts, nsamples, batch_size, stagmag_fxn, O_mat, sigx_fxn):
    
    # Load Model
    model,_,_,_ = loadmodel_fxn(path_to_config,path_to_ckpts)
    
    # Sample & calculate stag mags, sig xs
    if nsamples >= batch_size:
        batches = int((nsamples-(nsamples%batch_size))/batch_size)
        all_stagmags = []
        all_sigma_xs = []
        sigma_xs_per_site = np.zeros((1,256))
        for i in range(batches):
            print(f"Batch #{i}")
            samples,_ = model.sample(batch_size)
            stagmags,_,_,_ = stagmag_fxn(16,16,samples.numpy())
            all_stagmags.append(abs(stagmags))
            sigma_xs,_ = sigx_fxn(O_mat, samples, model.logpsi)
            all_sigma_xs.append(sigma_xs)

    # Take averages
    stagmag = np.mean(all_stagmags)
    sm_var = np.var(all_stagmags)
    sigma_x = np.mean(all_sigma_xs)
    sx_var = np.var(all_sigma_xs)
    
    return stagmag, sm_var, sigma_x, sx_var

In [15]:
sms_vmc1 = []
sms_vmc1_vars = []
sms_h1 = []
sms_h1_vars = []
sxs_vmc1 = []
sxs_vmc1_vars = []
sxs_h1 = []
sxs_h1_vars = []

sms_vmc2 = []
sms_vmc2_vars = []
sms_h2 = []
sms_h2_vars = []
sxs_vmc2 = []
sxs_vmc2_vars = []
sxs_h2 = []
sxs_h2_vars = []

scratch_path = '/scratch/msmoss/RNN_sims/Rydbergs/N_256/New_Results/'
train_quantities_path = scratch_path + 'train_quantities/'
interactions = buildlattice(16,16,trunc=100)
omega_matrix,_,_ = construct_mats(interactions,256)

for delta in deltas:
    print(delta)
    path_vmc1 = train_quantities_path + f'OneD_rnn/delta_{delta}/seed_111/vmc_only/'
    path_vmc1_ckpts = scratch_path + f'OneD_rnn/delta_{delta}/seed_111/vmc_only/'
    path_h1 = train_quantities_path + f'OneD_rnn/delta_{delta}/seed_111/hybrid_train/1000_ds/lr_5e-05/'
    path_h1_ckpts = scratch_path + f'OneD_rnn/delta_{delta}/seed_111/hybrid_train/1000_ds/lr_5e-05/'
    
    path_vmc2 = train_quantities_path + f'TwoD_rnn/delta_{delta}/seed_111/vmc_only/'
    path_vmc2_ckpts = scratch_path + f'TwoD_rnn/delta_{delta}/seed_111/vmc_only/'
    path_h2 = train_quantities_path + f'TwoD_rnn/delta_{delta}/seed_111/hybrid_train/100_ds/lr_0.001/'
    path_h2_ckpts = scratch_path + f'TwoD_rnn/delta_{delta}/seed_111/hybrid_train/100_ds/lr_0.001/'
    
    sm_vmc1, sm_var_vmc1, sx_vmc1, sx_var_vmc1 = calculate_observables(LoadModel,path_vmc1,path_vmc1_ckpts, 100000, 100, calculate_stag_mag, omega_matrix, calculate_sigma_x)
    sm_h1, sm_var_h1, sx_h1, sx_var_h1 = calculate_observables(LoadModel,path_h1,path_h1_ckpts, 100000, 100, calculate_stag_mag, omega_matrix, calculate_sigma_x)
    sm_vmc2, sm_var_vmc2, sx_vmc2, sx_var_vmc2 = calculate_observables(LoadModel,path_vmc2,path_vmc2_ckpts, 1000, 100, calculate_stag_mag, omega_matrix, calculate_sigma_x)
    sm_h2, sm_var_h2, sx_h2, sx_var_h2 = calculate_observables(LoadModel,path_h2,path_h2_ckpts, 1000, 100, calculate_stag_mag, omega_matrix, calculate_sigma_x)

    sms_vmc1.append(sm_vmc1)
    sms_vmc1_vars.append(sm_var_vmc1)
    sms_h1.append(sm_h1)
    sms_h1_vars.append(sm_var_h1)
    sxs_vmc1.append(sx_vmc1)
    sxs_vmc1_vars.append(sx_var_vmc1)
    sxs_h1.append(sx_h1)
    sxs_h1_vars.append(sx_var_h1)
    
    sms_vmc2.append(sm_vmc2)
    sms_vmc2_vars.append(sm_var_vmc2)
    sms_h2.append(sm_h2)
    sms_h2_vars.append(sm_var_h2)
    sxs_vmc2.append(sx_vmc2)
    sxs_vmc2_vars.append(sx_var_vmc2)
    sxs_h2.append(sx_h2)
    sxs_h2_vars.append(sx_var_h2)
    

-1.545
Training a two-D RNN wave function with 16 hidden units and shared weights = True.
The sum of params is 4674
Checkpoint found.
Restored from /scratch/msmoss/RNN_sims/Rydbergs/N_256/New_Results/TwoD_rnn/delta_-1.545/seed_111/vmc_only/ckpt-1000.
The final step was 10000.
Batch #0
Batch #1
Batch #2
Batch #3
Batch #4
Batch #5
Batch #6
Batch #7
Batch #8
Batch #9
Training a two-D RNN wave function with 16 hidden units and shared weights = True.
The sum of params is 4674
Checkpoint found.
Restored from /scratch/msmoss/RNN_sims/Rydbergs/N_256/New_Results/TwoD_rnn/delta_-1.545/seed_111/hybrid_train/100_ds/lr_0.001/ckpt-1000.
The final step was 10000.
Batch #0
Batch #1
Batch #2
Batch #3
Batch #4
Batch #5
Batch #6
Batch #7
Batch #8
Batch #9


AttributeError: 'numpy.float64' object has no attribute 'append'

# Plot

In [ ]:
plt.rcParams.update({"text.usetex":True,"font.family": "Helvetica"})

fig = plt.figure(1,figsize=(6,6), dpi=120, facecolor='w', edgecolor='k')
grid = plt.GridSpec(2, 1, wspace=0.4, hspace=0)
ax1 = fig.add_subplot(grid[0,0])
ax2 = fig.add_subplot(grid[1,0],sharex=ax1)
colors1 = sns.color_palette('pastel')
colors2 = sns.color_palette('dark')
d_o = deltas/4.24

# stag mags
ax1.errorbar(d_o, chks, yerr=chk_errors, marker='H', markerfacecolor='C3', markeredgecolor='C3', markersize=5, linewidth=0.5, alpha=0.6, linestyle='None', color='C3',label="QMC")
ax1.errorbar(d_o, load_stagmags2, yerr=np.sqrt(load_stagmags2_vars)/np.sqrt(10000), marker='o', markerfacecolor=colors2[0], markeredgecolor=colors2[0], markersize=5, linewidth=0.5, alpha=0.6, linestyle='None', color=colors2[0],label ="VMC, 2D RNN")
ax1.errorbar(d_o, hybrid_stagmags2, yerr=np.sqrt(hybrid_stagmags2_vars)/np.sqrt(10000), marker='d', markerfacecolor=colors2[2], markeredgecolor=colors2[2], markersize=5, linewidth=0.5, alpha=0.6, linestyle='None', color=colors2[2],label ="Hybrid, 2D RNN")
ax1.errorbar(d_o, load_stagmags1, yerr=np.sqrt(load_stagmags1_vars)/np.sqrt(10000), marker='o', markerfacecolor=colors1[0], markeredgecolor=colors1[0], markersize=5, linewidth=0.5, alpha=0.6, linestyle='None', color=colors1[0],label ="VMC, 1D RNN")
ax1.errorbar(d_o, exp_data_stagmags, yerr=np.sqrt(exp_data_vars)/np.sqrt(1000), marker='s', markerfacecolor='C1', markeredgecolor='C1', markersize=5, linewidth=0.6, alpha=0.5, linestyle='None', color='C1',label="Experimental Data")
ax1.errorbar(d_o, hybrid_stagmags1, yerr=np.sqrt(hybrid_stagmags1_vars)/np.sqrt(10000), marker='d', markerfacecolor=colors1[2], markeredgecolor=colors1[2], markersize=5, linewidth=0.6, alpha=0.5, linestyle='None', color=colors1[2],label ="Hybrid, 1D RNN")
ax1.vlines(4.765/4.24,0.001,0.45,linestyle='--',color='grey',alpha=0.75,label ='$\\delta_c/\\Omega$')
ax1.set_ylabel('$\\langle M_{\mathrm{s}} \\rangle/N$',fontsize=20)
ax1.text(-1.05, 0.45, "(a)",fontsize=15)

    
# sigma x
ax2.errorbar(d_o, load_sigxs2, yerr=np.sqrt(load_sigxs2_vars)/np.sqrt(10000), marker='o', markerfacecolor=colors2[0], markeredgecolor=colors2[0], markersize=5, linewidth=0.5, alpha=0.6, linestyle='None', color=colors2[0])
ax2.errorbar(d_o, hybrid_sigxs2, yerr=np.sqrt(hybrid_sigxs2_vars)/np.sqrt(10000), marker='d', markerfacecolor=colors2[2], markeredgecolor=colors2[2], markersize=5, linewidth=0.6, alpha=0.5, linestyle='None', color=colors2[2])
ax2.errorbar(d_o, load_sigxs1, yerr=np.sqrt(load_sigxs1_vars)/np.sqrt(10000), marker='o', markerfacecolor=colors1[0], markeredgecolor=colors1[0], markersize=5, linewidth=0.5, alpha=0.6, linestyle='None', color=colors1[0])
ax2.errorbar(d_o, hybrid_sigxs, yerr=np.sqrt(hybrid_sigxs_vars)/np.sqrt(10000), marker='d', markerfacecolor=colors1[2], markeredgecolor=colors1[2], markersize=5, linewidth=0.6, alpha=0.5, linestyle='None', color=colors1[2])
ax2.vlines(4.765/4.24,0.25,0.8,linestyle='--',color='grey',alpha=0.75)
ax2.set_ylabel('$\\langle \\sigma_{\mathrm{x}} \\rangle/N$',fontsize=20)
ax2.set_ylim(0.2,0.85)
ax2.text(-1.05, 0.78, "(b)",fontsize=15)

fig.legend(loc='center', bbox_to_anchor=(0.28, 0.75))
plt.xlabel('$ \delta/\\Omega $',fontsize=20)
plt.show()